<a href="https://colab.research.google.com/github/spiritofdragon/lung_tumor_segmentation/blob/main/lung_tumor_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install celluloid

In [1]:
%matplotlib notebook
from pathlib import Path # Path
import nibabel as nib # as the file are provided in the NIfTi format
import numpy as np # data saving or processing
import matplotlib.pyplot as plt # to show the flotting
from tqdm.notebook import tqdm # to show progress bar
import cv2

In [2]:
root = Path("/content/drive/MyDrive/07-Lung-Tumor/imagesTr")
label = Path("/content/drive/MyDrive/07-Lung-Tumor/labelsTr")

In [3]:
def change_image_to_label_path(path): # path: list(root.glob("lu*"))[0]
    parts = list(path.parts) #
    parts[parts.index("imagesTr")] = "labelsTr"
    return Path(*parts)

In [4]:
sample_path = list(root.glob("lung*"))[0] # direct to Task06_lung/imagesTr get the name begin with "lu*" (use glob)
sample_path_label = change_image_to_label_path(sample_path)

In [5]:
sample_path, sample_path_label

(PosixPath('/content/drive/MyDrive/07-Lung-Tumor/imagesTr/lung_001.nii.gz'),
 PosixPath('/content/drive/MyDrive/07-Lung-Tumor/labelsTr/lung_001.nii.gz'))

In [6]:
data = nib.load(sample_path) # load NIfTi file format
label = nib.load(sample_path_label)

ct = data.get_fdata() #to retrieve the underlying data as a NumPy array from objects that store volumetric or multidimensional data, such as medical imaging data stored in NIfTI format
mask = label.get_fdata().astype(np.uint8)

In [9]:
from celluloid import Camera

In [10]:
nib.aff2axcodes(data.affine)

('L', 'A', 'S')

In [11]:
ct.shape

(512, 512, 304)

In [14]:
fig = plt.figure()
camera = Camera(fig)

for i in range(0, ct.shape[2], 2):
    plt.imshow(ct[:,:,i], cmap="bone") # ct[width,heigh,slice number i]
    mask_ = np.ma.masked_where(mask[:,:,i]==0, mask[:,:,i])
    plt.imshow(mask_, alpha=0.3, cmap="autumn") # do trong suot cua mask_ so voi ct
    camera.snap()
animation = camera.animate()

<IPython.core.display.Javascript object>

In [15]:
from IPython.display import HTML # or: animation.save('animation.mp4')
HTML(animation.to_html5_video())

In [16]:
all_files = list(root.glob("lung_*"))  # Get all subjects
len(all_files)

63

In [17]:
def normalize(full_volume):
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized

def standardize(normalized_data):
    standardized_data = (normalized_data - normalized_data.min())/(normalized_data.max() - normalized_data.min())
    return standardized_data

In [18]:
def normalize(full_volume):
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized

def standardize(normalized):
    standardized = (normalized - normalized.min())/(normalized.max() - normalized.min())
    return standardized

In [ ]:
save_root = Path("Preprocessed")

for counter, path_to_ct_data in enumerate(tqdm(all_files)): #to iterate over all_files and provides an index (counter) and the corresponding element (path_to_mri_data) in each iteration

    path_to_label = change_image_to_label_path(path_to_ct_data)

    ct = nib.load(path_to_ct_data)

    assert nib.aff2axcodes(ct.affine) == ("L", "A", "S")

    ct_data = ct.get_fdata()
    label_data = nib.load(path_to_label).get_fdata()

    ct_data = ct_data[:,:,30:]/3071
    label_data = label_data[:,:,30:]

    normalized_ct_data = normalize(ct_data)
    standardized_ct_data = standardize(normalized_ct_data)

    if counter < 57:
        current_path = save_root/"train"/str(counter)
    else:
        current_path = save_root/"val"/str(counter)

    for i in range(standardized_ct_data.shape[-1]):
        slice = standardized_ct_data[:,:,i]
        mask = label_data[:,:,i]

        # Resize slice and label to common resolution to reduce training time
        slice = cv2.resize(slice, (256, 256))
        mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)

        slice_path = current_path/"data"
        mask_path = current_path/"masks"
        slice_path.mkdir(parents=True, exist_ok=True)
        mask_path.mkdir(parents=True, exist_ok=True)

        np.save(slice_path/str(i), slice)
        np.save(mask_path/str(i), mask)

  0%|          | 0/63 [00:00<?, ?it/s]